# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os import listdir
folderPath = '/content/drive/MyDrive/Project/finetune data'
dataPath = '/content/drive/MyDrive/Project/Fantasy Novels/'
for bookName in listdir(dataPath):
    print(bookName)

Witcher
TLOR
GOT


# Run Ollama in Colab

In [ ]:
# 安裝能開啟終端機的插件
!pip install colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
# 開啟終端機
%xterm
'''接下來請在終端機裡分別執行以下三行指令
  curl https://ollama.ai/install.sh | sh
  ollama serve &
  在跑出 name='Tesla T4'的一行後才代表ollama在背景成功執行 這時才繼續貼下一行
  ollama pull llama3.1:8b
'''

# 兩行分別的功能是
# curl https://ollama.ai/install.sh | sh: 從官網下載並安裝ollama
# ollama serve & ollama pull llama3.1:8b: 這其實是兩行指令分成
# ollama serve &: 啟動ollama並在背景執行
# ollama pull llama3.1:8b:使用pull關鍵字透過網路下載指定模型
# 嘗試不同模型可以去https://ollama.com/library找

Launching Xterm...

<IPython.core.display.Javascript object>

"接下來請在終端機裡分別執行以下三行指令\n  curl https://ollama.ai/install.sh | sh\n  ollama serve &\n  在跑出 name='Tesla T4'的一行後才代表ollama在背景成功執行 這時才繼續貼下一行\n  ollama pull llama3.1:8b\n"

In [ ]:
!pip install ollama
!pip install tqdm
!ollama list

NAME           ID              SIZE      MODIFIED       
llama3.1:8b    42182419e950    4.7 GB    10 seconds ago    


應該要印出以下結果

NAME           ID              SIZE      MODIFIED      
llama3.1:8b    42182419e950    4.7 GB    8 seconds ago

# Ollama Functions
給定prompt 用以下指令呼叫ollama下載好的模型

In [ ]:
# 如果要換模型記得上面下載模型也要跟著改 這裡只改呼叫的名稱
language_model="llama3.1:8b"

# Instruction Generation
Tell ollama to do as follows:


In [ ]:
import ollama
from tqdm import tqdm # 視覺化進度條的插件
import json       # 生成json檔
import re        # Regular Expression 正規語言，用來將模型輸出做Parse

def generate_two_content_response(model, prompt, story_text1,story_text2):
  # Please strictly follow the input prompt as follows:
  # """....{text1}....{text2}...."""
  response = ollama.generate(
        model = model,
        prompt = prompt.format(text1=story_text1,text2=story_text2))
  output = response['response']
  return output

def string_to_dict(s):
    # Define the pattern to capture only the "instruction" field
    pattern = r'"instruction":\s*"([^"]+)"'

    # Search for the "instruction" field in the input string
    match = re.search(pattern, s, re.DOTALL)

    # If found, return a dictionary with only the "instruction" key
    if match:
        return {
            "instruction": match.group(1).strip()
        }

    # Return an empty dictionary if no match is found
    return {}

def process_data_by_prompt(story_list,prompt, num_per_data=1):
  # 一次處理一個list的函式 回傳 string list

  output_list = []
  num_story=len(story_list)
  for story_idx in tqdm(range(1,num_story)):
    # 因為要index story_index-1所以從1開始
    for _ in range(num_per_data):
      # output_dict = string_to_dict(generate_response(language_model, prompt,story)) 這是使用單段故事生成時用的指令 如summary類指令
      output_dict = string_to_dict(generate_two_content_response(language_model, prompt, story_list[story_idx-1],story_list[story_idx]))
      if output_dict:
        output_dict['input']=story_list[story_idx-1]
        output_dict['output']=story_list[story_idx]
        output_list.append(output_dict)
  return output_list

In [ ]:
inst_prompt = '''
    Based on the following text, generate 1 high-quality entry for an instruction dataset. The entry should directly relate to the given text content and pose a relevant question or task.
    :

    Text content 1:
    {text1}

    Text content 2:
    {text2}

    Please format the entry as follows, making sure each field has appropriate content:
    {{
        "instruction": "Clear, plot-specific instruction that describes how the input leads to the output and the intended emotional or narrative development"
    }}
    Ensure all generated content is directly related to the given text, formatted as valid JSON, and is high-quality, accurate, and detailed.
    '''

In [ ]:
import os
def list_of_dicts_to_json(dict_list, file_name='output.json', directory=folderPath):
    # Define the full path for the file
    file_path = os.path.join(directory, file_name)

    # Convert the list of dictionaries to JSON and save to file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(dict_list, f, ensure_ascii=False, indent=4)

    print(f"JSON file saved to {file_path}")


In [ ]:
import json       # 生成json檔
import re        # Regular Expression 正規語言，用來將模型輸出做Parse
file_path = folderPath + "/string_list.json"
# Read the list back from the file
with open(file_path, "r") as file:
    story_list = json.load(file)

print(len(story_list))


2521


# CHANGE THE INDEX HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
## 改這裡INDEX!!!!!!!!!!!!!!!!!

In [ ]:
from tqdm import tqdm
###########################################
# Change the index here
start_idx=13
###########################################
num_per_output=200
num_per_data=1
for idx in range(start_idx,len(story_list)//num_per_output+1):
  output_list = process_data_by_prompt(story_list[idx*num_per_output:(idx+1)*num_per_output], inst_prompt, num_per_data=num_per_data)
  print(f'Success Rate: {len(output_list)/(num_per_data*len(story_list)):.2f}')
  list_of_dicts_to_json(output_list, file_name='output_'+str(idx)+'.json', directory=folderPath)


# 合成多份json

## 自己不用跑，之後等大家跑完會再跑
## No need to run, we'll run after everyone finishes

In [ ]:
import json
import glob
import os
from pathlib import Path

directory = folderPath

combined_data = []
json_pattern = os.path.join(directory, "output_*.json")
file_list = glob.glob(json_pattern)

try:
    # Read each file and append its content to combined_data
    for file_path in file_list:
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)
                # If the file contains a list, extend combined_data
                if isinstance(data, list):
                    combined_data.extend(data)
                # If the file contains a single object, append it
                else:
                    combined_data.append(data)
                print(f"Successfully processed: {file_path}, which contains {len(data)} elements")
            except json.JSONDecodeError as e:
                print(f"Error reading {file_path}: {str(e)}")
                continue

    # Write the combined data to a new file
    output_path = os.path.join(directory, "combined_output.json")
    with open(output_path, 'w', encoding='utf-8') as outfile:
        json.dump(combined_data, outfile, indent=2, ensure_ascii=False)

    print(f"\nSuccessfully combined {len(file_list)} files into: {output_path}")
    print(f"Total number of entries: {len(combined_data)}")

except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
filter_words_file = open("/content/drive/MyDrive/Project/filter_words.json", "r", encoding="utf-8")
filter_words = json.load(filter_words_file)

def filter(json_object, filter_words=filter_words):
  filtered = []
  for idx in range(len(json_object)):
    flag = True
    for filter_word in filter_words:
      if filter_word in json_object[idx]["instruction"]:
        flag = False
        break
    if flag:
      filtered.extend([json_object[idx]])
  return filtered

In [ ]:
with open(folderPath + "/output_3.json", 'r', encoding='utf-8') as file:
  data = json.load(file)
  # If the file contains a list, extend combined_data
  if isinstance(data, list):
    print(filter(data))

[{'instruction': 'The druids in Myrkvid Forest will only stop their impending execution of a group of bandits and a half-elf if Geralt can persuade them that one of the bandits has crucial information for him, but the flaminika is unmoved by his pleas and will not show mercy to those who have broken the law.', 'input': '‘’Pon my word!’ crowed the other, with a uniformly blue shield, in a youthful voice. ‘We’ll carve them up on the spot!’\n‘Hi, I say! Stop!’\nThe Chequered Knight emerged from the fog on Bucephalus. He had finally managed to lift his twisted visor, from beneath which luxuriant flaxen moustaches now peeped.\n‘Free them with all haste!’ he called. ‘They are not bandits, but upright and honest folk. The lady manfully acted in defence of the pilgrims. And that fellow is a goodly knight!’\n‘A goodly knight?’ Bull’s Head raised his visor and scrutinised Geralt extremely incredulously. ‘’Pon my word! It cannot be!’\n‘’Pon my word!’ The Chequered Knight thumped an armoured fist 